<a href="https://colab.research.google.com/github/anushhhh/pitch-recommendation-algorithm/blob/main/pitch_recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
post_data = pd.read_csv('feeds.csv', on_bad_lines='skip')
view_data = pd.read_csv('userinteractions.csv', on_bad_lines='skip')
user_data = pd.read_csv('users.csv', on_bad_lines='skip')

<ipython-input-2-e4e6fc645fe7>:2: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,

In [3]:
post_data.shape

(777, 22)

In [4]:
post_data.head()

,_id,feed_id,__v,active,createdDate,description,modifiedDate,postCommentCount,postImageURLs,postMessage,...,postReportCount,postRunCount,sharedPost,type,userId,userName,userProfilePhoto,processed,impression,sharedDetail
0,"{""$oid"":""64b92a83fc27b8ebe9da5f8f""}","{""$oid"":""64b8fe03f83400d2a74a9dff""}",0,True,"{""$date"":""2023-07-20T09:27:31.039Z""}","{""hashtag"":[]}","{""$date"":""2023-07-21T06:19:11.514Z""}",2.0,"[""https://chamhunt-file-save-app.s3.us-west-1....",AB de Villiers full of praise for Yashasvi Jai...,...,0.0,6.0,"[""63104b7c35de2d1ef088e986""]",post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,False,NaN,NaN
1,"{""$oid"":""64b92b56fc27b8ebe9da7f1f""}","{""$oid"":""64b18f463f563c7233541165""}",0,True,"{""$date"":""2023-07-14T18:09:10.076Z""}",NaN,"{""$date"":""2023-10-31T06:10:54.821Z""}",0.0,[],super,...,0.0,0.0,[],post,63ad12a49145dce3996a3f17,jignesh j,https://chamhunt-file-save-app.s3.us-west-1.am...,False,1.0,NaN
2,"{""$oid"":""64b92b56fc27b8ebe9da7f36""}","{""$oid"":""64b178b73f563c7233540f24""}",0,True,"{""$date"":""2023-07-14T16:32:55.836Z""}","{""hashtag"":[]}","{""$date"":""2023-07-14T16:32:55.836Z""}",0.0,"[""https://chamhunt-file-save-app.s3.us-west-1....",Virat Kohli went ahead of Virender Sehwag in t...,...,0.0,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN
3,"{""$oid"":""64b92b56fc27b8ebe9da7f42""}","{""$oid"":""64b1a2f83f563c7233541347""}",0,True,"{""$date"":""2023-07-14T19:33:12.119Z""}","{""hashtag"":[]}","{""$date"":""2023-07-14T19:33:12.120Z""}",0.0,"[""https://chamhunt-file-save-app.s3.us-west-1....",Ruturaj Gaikwad has been announced as the skip...,...,0.0,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN
4,"{""$oid"":""64b92b56fc27b8ebe9da7f4a""}","{""$oid"":""64b178963f563c7233540eef""}",0,True,"{""$date"":""2023-07-14T16:32:22.822Z""}","{""hashtag"":[]}","{""$date"":""2023-07-14T16:32:22.822Z""}",0.0,"[""https://chamhunt-file-save-app.s3.us-west-1....",Yashasvi Jaiswal has registered a 140+ score o...,...,0.0,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN


In [5]:
def extract_nested_data(cell):
    try:
        # Parse the cell if it's a JSON-like string
        parsed = json.loads(cell)

        # Handle dictionary case
        if isinstance(parsed, dict):
            return parsed.get('$oid') or parsed.get('$date') or parsed

        # Handle list case
        if isinstance(parsed, list):
            return [extract_nested_data(str(item)) for item in parsed]

        # Handle primitive types
        return parsed
    except (ValueError, TypeError):
        return cell

for column in post_data.columns:
    post_data[column] = post_data[column].apply(extract_nested_data)

for column in user_data.columns:
    user_data[column] = user_data[column].apply(extract_nested_data)

for column in view_data.columns:
    view_data[column] = view_data[column].apply(extract_nested_data)

In [6]:
# Filter out posts with empty or NaN postMessage
post_data = post_data[post_data['postMessage'].notna() & (post_data['postMessage'].str.strip() != '')]

# Reset index after filtering
post_data.reset_index(drop=True, inplace=True)

In [7]:
post_data.shape

(298, 22)

In [8]:
user_data.head()

,_id,firstName,lastName,following,followers,followersCount,active,aboutMe,state,DOB,Gender
0,62460411a1185e0d52b13c69,Vish,Aggarwal,[{'followingUserId': '62460453a1185e0d52b13c83...,[{'followerUserId': '62460453a1185e0d52b13c83'...,NaN,True,Hi!,Uttarakhand,NaN,NaN
1,62460584a1185e0d52b13e1e,CHAMPHUNT,LLC,[{'followingUserId': '6246842ba1185e0d52b14468...,[{'followerUserId': '62460453a1185e0d52b13c83'...,72211.0,True,"The Official page of Champhunt, your home for ...",NaN,NaN,NaN
2,62460453a1185e0d52b13c83,Sameer,Kanva,[{'followingUserId': '62460411a1185e0d52b13c69...,[{'followerUserId': '62460411a1185e0d52b13c69'...,613.0,True,"Founder, CEO, Champhunt INC\r\n\r\nI just don'...",Maharashtra,1987-12-18,Male
3,62467e7ba1185e0d52b14108,Monali,M,[{'followingUserId': '624732d1a1185e0d52b2b3fd...,[{'followerUserId': '6263a270de3b355340c8a415'...,51.0,True,NaN,NaN,NaN,NaN
4,62469625a1185e0d52b15220,KKR Fans,Club,[{'followingUserId': '62460584a1185e0d52b13e1e...,[{'followerUserId': '62468320a1185e0d52b1424b'...,NaN,True,"Aami Kolkata, We rule!\r\n\r\nFan Page of KKR.",West Bengal,NaN,NaN


In [9]:
from datetime import datetime

def calculate_age(dob):
    if pd.notna(dob):
        dob = pd.to_datetime(dob)
        return datetime.now().year - dob.year
    return None

# Create age column in user_data
user_data['age'] = user_data['DOB'].apply(calculate_age)

# Define age groups
def get_age_group(age):
    if age is None or np.isnan(age):  # Check for None or NaN
        return 'unknown'
    elif age < 18:
        return 'teen'
    elif 18 <= age < 30:
        return 'young_adult'
    elif 30 <= age < 50:
        return 'adult'
    else:
        return 'senior'

user_data['age_group'] = user_data['age'].apply(get_age_group)

In [10]:
view_data.head()

,_id,userId,interactions[0].tagId,interactions[1].tagId,interactions[2].tagId,interactions[3].tagId,interactions[4].tagId,interactions[5].tagId,interactions[6].tagId,interactions[7].tagId,...,interactions[784]._id,interactions[785]._id,interactions[786]._id,interactions[787]._id,location.latitude,location.longitude,location.source,createdAt,updatedAt,__v
0,679b4e9b74c475998774e340,6246842ba1185e0d52b14468,679b4e9b74c475998774e33d,679b4e9b74c475998774e344,678f942bc798d2b5de45998d,679b4e9b74c475998774e352,679b4e9b74c475998774e35b,679b4e9b74c475998774e365,678f942bc798d2b5de459e35,679b4e9d74c475998774e3a5,...,NaN,NaN,NaN,NaN,0,0,ip,2025-01-30T10:04:11.110Z,2025-01-30T10:36:05.473Z,157
1,679b4e9e74c475998774e3f7,62460453a1185e0d52b13c83,679b4e9b74c475998774e33d,678f942bc798d2b5de45998d,679b4e9e74c475998774e401,678f942bc798d2b5de459e35,679b4e9b74c475998774e344,679b4e9e74c475998774e41a,679b4e9b74c475998774e352,679b4e9d74c475998774e3a5,...,NaN,NaN,NaN,NaN,0,0,ip,2025-01-30T10:04:14.225Z,2025-01-30T11:05:16.947Z,241
2,679b4e9e74c475998774e43d,62468320a1185e0d52b1424b,679b4e9b74c475998774e33d,678f942bc798d2b5de45998d,679b4e9d74c475998774e3c0,679b4e9f74c475998774e44e,678f942bc798d2b5de45a727,679b4ea074c475998774e4e5,678f942bc798d2b5de45ab51,679b4e9f74c475998774e476,...,NaN,NaN,NaN,NaN,0,0,ip,2025-01-30T10:04:14.788Z,2025-01-30T10:33:48.657Z,83
3,679b4e9f74c475998774e479,624940020025a06338458dac,679b4e9f74c475998774e476,679b4e9f74c475998774e47d,679b4e9f74c475998774e484,679b4e9b74c475998774e33d,679b3af28b4c8301ca16a6ba,679b4ea274c475998774e580,678f942bc798d2b5de4598af,678f942bc798d2b5de45ab23,...,NaN,NaN,NaN,NaN,0,0,ip,2025-01-30T10:04:15.570Z,2025-01-30T10:05:19.925Z,16
4,679b4e9f74c475998774e494,6249818d0025a0633846b3fb,679b4e9b74c475998774e33d,679b4e9f74c475998774e498,679b4e9f74c475998774e49f,679b4e9f74c475998774e4a7,678f96d3c798d2b5de5a05d7,678f942bc798d2b5de459a71,678f942bc798d2b5de459e68,679b4ea074c475998774e4ce,...,NaN,NaN,NaN,NaN,0,0,ip,2025-01-30T10:04:15.770Z,2025-01-30T10:04:16.140Z,7


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD

In [12]:
# Fill missing values and convert all values to strings
post_data['postMessage'] = post_data['postMessage'].fillna('').apply(str)

# Apply TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english')
post_vectors = vectorizer.fit_transform(post_data['postMessage'].fillna(''))

In [13]:
def is_valid_post(message):
    # Check for minimum length and valid alphanumeric content
    return bool(re.search(r'[a-zA-Z0-9]', message)) and len(message.strip()) > 5

In [14]:
# Filter posts with meaningful post messages
post_data = post_data[post_data['postMessage'].apply(is_valid_post)]
post_data.reset_index(drop=True, inplace=True)

post_vectors = vectorizer.fit_transform(post_data['postMessage'])

In [15]:
post_data

,_id,feed_id,__v,active,createdDate,description,modifiedDate,postCommentCount,postImageURLs,postMessage,...,postReportCount,postRunCount,sharedPost,type,userId,userName,userProfilePhoto,processed,impression,sharedDetail
0,64b92a83fc27b8ebe9da5f8f,64b8fe03f83400d2a74a9dff,0,True,2023-07-20T09:27:31.039Z,{'hashtag': []},2023-07-21T06:19:11.514Z,2.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,AB de Villiers full of praise for Yashasvi Jai...,...,0.0,6.0,[63104b7c35de2d1ef088e986],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,False,NaN,NaN
1,64b92b56fc27b8ebe9da7f36,64b178b73f563c7233540f24,0,True,2023-07-14T16:32:55.836Z,{'hashtag': []},2023-07-14T16:32:55.836Z,0.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,Virat Kohli went ahead of Virender Sehwag in t...,...,0.0,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN
2,64b92b56fc27b8ebe9da7f42,64b1a2f83f563c7233541347,0,True,2023-07-14T19:33:12.119Z,{'hashtag': []},2023-07-14T19:33:12.120Z,0.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,Ruturaj Gaikwad has been announced as the skip...,...,0.0,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN
3,64b92b56fc27b8ebe9da7f4a,64b178963f563c7233540eef,0,True,2023-07-14T16:32:22.822Z,{'hashtag': []},2023-07-14T16:32:22.822Z,0.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,Yashasvi Jaiswal has registered a 140+ score o...,...,0.0,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN
4,64b92b56fc27b8ebe9da7f56,64b15ac7f83400d2a74a310e,0,True,2023-07-14T14:25:11.253Z,{'hashtag': []},2023-07-14T14:45:21.574Z,0.0,[],Yashasvi Jaiswal showing some great game!! 150...,...,0.0,6.0,[],post,6247c5e80025a0633841ac54,Ansh Desai,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,64b92b5afc27b8ebe9da90ad,6495b325611bcbb03d999972,0,False,2023-06-23T14:58:45.515Z,NaN,2023-06-24T10:37:07.152Z,0.0,[],djjdjd,...,0.0,12.0,[],post,6495b00bfd0018f32c9db637,Par Saini,https://lh3.googleusercontent.com/a/AAcHTtd1tg...,NaN,NaN,NaN
135,64b92b5afc27b8ebe9da90a2,6495b2f6fd0018f32c9dbeba,0,False,2023-06-23T14:57:58.829Z,NaN,2023-06-24T10:38:20.024Z,0.0,[],hdjdjd,...,0.0,12.0,[],post,6495b00bfd0018f32c9db637,Par Saini,https://lh3.googleusercontent.com/a/AAcHTtd1tg...,NaN,NaN,NaN
136,64b92b5afc27b8ebe9da9125,6495bbdd611bcbb03d99a512,0,False,2023-06-23T15:35:57.108Z,NaN,2023-06-24T10:36:55.321Z,0.0,[],eg te d,...,0.0,18.0,[],post,6495b00bfd0018f32c9db637,Par Saini,https://lh3.googleusercontent.com/a/AAcHTtd1tg...,NaN,NaN,NaN
137,64b92b5afc27b8ebe9da9275,6495875d611bcbb03d9970a9,0,True,2023-06-23T11:51:57.978Z,NaN,2023-06-23T11:51:57.978Z,0.0,[],Rishabh pant,...,0.0,0.0,[],post,647024071e0424f9f8174d38,Parth_7uhh Instagram.,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN


In [16]:
post_data['popularity_score'] = (
    0.4 * post_data['postCommentCount'].fillna(0) +
    0.3 * post_data['postRunCount'].fillna(0) +
    0.3 * post_data['impression'].fillna(0)
)

In [17]:
post_data

,_id,feed_id,__v,active,createdDate,description,modifiedDate,postCommentCount,postImageURLs,postMessage,...,postRunCount,sharedPost,type,userId,userName,userProfilePhoto,processed,impression,sharedDetail,popularity_score
0,64b92a83fc27b8ebe9da5f8f,64b8fe03f83400d2a74a9dff,0,True,2023-07-20T09:27:31.039Z,{'hashtag': []},2023-07-21T06:19:11.514Z,2.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,AB de Villiers full of praise for Yashasvi Jai...,...,6.0,[63104b7c35de2d1ef088e986],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,False,NaN,NaN,2.6
1,64b92b56fc27b8ebe9da7f36,64b178b73f563c7233540f24,0,True,2023-07-14T16:32:55.836Z,{'hashtag': []},2023-07-14T16:32:55.836Z,0.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,Virat Kohli went ahead of Virender Sehwag in t...,...,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN,0.0
2,64b92b56fc27b8ebe9da7f42,64b1a2f83f563c7233541347,0,True,2023-07-14T19:33:12.119Z,{'hashtag': []},2023-07-14T19:33:12.120Z,0.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,Ruturaj Gaikwad has been announced as the skip...,...,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN,0.0
3,64b92b56fc27b8ebe9da7f4a,64b178963f563c7233540eef,0,True,2023-07-14T16:32:22.822Z,{'hashtag': []},2023-07-14T16:32:22.822Z,0.0,[https://chamhunt-file-save-app.s3.us-west-1.a...,Yashasvi Jaiswal has registered a 140+ score o...,...,0.0,[],post,6489d682611bcbb03d9411b8,cricket.com official,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN,0.0
4,64b92b56fc27b8ebe9da7f56,64b15ac7f83400d2a74a310e,0,True,2023-07-14T14:25:11.253Z,{'hashtag': []},2023-07-14T14:45:21.574Z,0.0,[],Yashasvi Jaiswal showing some great game!! 150...,...,6.0,[],post,6247c5e80025a0633841ac54,Ansh Desai,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,64b92b5afc27b8ebe9da90ad,6495b325611bcbb03d999972,0,False,2023-06-23T14:58:45.515Z,NaN,2023-06-24T10:37:07.152Z,0.0,[],djjdjd,...,12.0,[],post,6495b00bfd0018f32c9db637,Par Saini,https://lh3.googleusercontent.com/a/AAcHTtd1tg...,NaN,NaN,NaN,3.6
135,64b92b5afc27b8ebe9da90a2,6495b2f6fd0018f32c9dbeba,0,False,2023-06-23T14:57:58.829Z,NaN,2023-06-24T10:38:20.024Z,0.0,[],hdjdjd,...,12.0,[],post,6495b00bfd0018f32c9db637,Par Saini,https://lh3.googleusercontent.com/a/AAcHTtd1tg...,NaN,NaN,NaN,3.6
136,64b92b5afc27b8ebe9da9125,6495bbdd611bcbb03d99a512,0,False,2023-06-23T15:35:57.108Z,NaN,2023-06-24T10:36:55.321Z,0.0,[],eg te d,...,18.0,[],post,6495b00bfd0018f32c9db637,Par Saini,https://lh3.googleusercontent.com/a/AAcHTtd1tg...,NaN,NaN,NaN,5.4
137,64b92b5afc27b8ebe9da9275,6495875d611bcbb03d9970a9,0,True,2023-06-23T11:51:57.978Z,NaN,2023-06-23T11:51:57.978Z,0.0,[],Rishabh pant,...,0.0,[],post,647024071e0424f9f8174d38,Parth_7uhh Instagram.,https://chamhunt-file-save-app.s3.us-west-1.am...,NaN,NaN,NaN,0.0


In [18]:
scaler = MinMaxScaler()
post_data['popularity_score'] = scaler.fit_transform(post_data[['popularity_score']])

In [19]:
# User content profile (sum of vectors from viewed posts)
def get_user_content_profile(user_id):
    user_views = view_data[view_data['userId'] == user_id]
    viewed_post_ids = user_views['_id'].tolist()
    viewed_vectors = post_vectors[post_data['_id'].isin(viewed_post_ids)]
    if viewed_vectors.shape[0] == 0:
        return np.zeros((1, post_vectors.shape[1]))
    return viewed_vectors.mean(axis=0)

In [20]:
# Step 2: Collaborative filtering via interaction matrix
interaction_matrix = view_data.pivot(index='userId', columns='_id', values='__v').fillna(0)

In [21]:
interaction_matrix

_id,679b4e9b74c475998774e340,679b4e9e74c475998774e3f7,679b4e9e74c475998774e43d,679b4e9f74c475998774e479,679b4e9f74c475998774e494,679b4ea274c475998774e576,679b4ea874c475998774e874,679b4eac74c475998774e98e,679b4ead74c475998774e9a3,679b4eb674c475998774f1ab,...,679b60b13c434fe2bdd726ea,679b60b53c434fe2bdd72af7,679b60ba3c434fe2bdd73081,679b60be3c434fe2bdd74018,679b60be3c434fe2bdd7401d,679b60bf3c434fe2bdd740f8,679b60bf3c434fe2bdd74101,679b60c03c434fe2bdd7428e,679b60e23c434fe2bdd74f6f,679b611b3c434fe2bdd75aa0
userId,,,,,,,,,,,,,,,,,,,,,
62460411a1185e0d52b13c69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62460453a1185e0d52b13c83,0.0,241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62460584a1185e0d52b13e1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62468320a1185e0d52b1424b,0.0,0.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6246842ba1185e0d52b14468,157.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794b12e2f9953e3d1cbbe9b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
679606dc2f9953e3d1da80c2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67964f4b2f9953e3d1dea8df,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Collaborative similarity
svd = TruncatedSVD(n_components=50)
user_factors = svd.fit_transform(interaction_matrix)

In [23]:
def recommend_posts(user_id, top_n=5):
    # Get user content profile and similarity scores
    user_profile = get_user_content_profile(user_id)
    content_scores = cosine_similarity(user_profile, post_vectors).flatten()

    # Ensure post IDs alignment between post_data and interaction_matrix
    post_ids = post_data['_id'].tolist()

    # Collaborative filtering scores
    collab_scores = np.zeros(len(post_ids))  # Default to zeros for unmatched posts

    if user_id in interaction_matrix.index:
        user_index = interaction_matrix.index.get_loc(user_id)
        user_collab_scores = user_factors[user_index]

        # Map collaborative scores to post indices in post_data
        collab_score_dict = dict(zip(interaction_matrix.columns, user_collab_scores))

        # Assign collaborative scores where post IDs match
        collab_scores = np.array([collab_score_dict.get(post_id, 0) for post_id in post_ids])

    # Combine scores (weighted)
    combined_scores = 0.5 * content_scores + 0.5 * collab_scores

    # Rank and recommend posts
    recommended_indices = np.argsort(combined_scores)[-top_n:][::-1]
    return post_data.iloc[recommended_indices][['_id', 'postMessage']]

**ALGORITHM WITH MOST ENGAGED POSTS AS RECOMMENDATIONS**

In [23]:
def recommend_posts(user_id, top_n=5):
    # Content and collaborative scores
    user_profile = get_user_content_profile(user_id)
    content_scores = cosine_similarity(user_profile, post_vectors).flatten()

    post_ids = post_data['_id'].tolist()
    collab_scores = np.zeros(len(post_ids))  # Default for unmatched posts

    if user_id in interaction_matrix.index:
        user_index = interaction_matrix.index.get_loc(user_id)
        user_collab_scores = user_factors[user_index]
        collab_score_dict = dict(zip(interaction_matrix.columns, user_collab_scores))
        collab_scores = np.array([collab_score_dict.get(post_id, 0) for post_id in post_ids])

    # Normalize scores
    content_scores = content_scores / content_scores.max() if content_scores.max() > 0 else content_scores
    collab_scores = collab_scores / collab_scores.max() if collab_scores.max() > 0 else collab_scores

    # Weighted combination with popularity
    popularity_scores = post_data['popularity_score'].values
    combined_scores = 0.4 * content_scores + 0.3 * collab_scores + 0.3 * popularity_scores

    # Rank and recommend posts
    recommended_indices = np.argsort(combined_scores)[-top_n:][::-1]
    return post_data.iloc[recommended_indices][['_id', 'postMessage', 'popularity_score']]

**ALGORITM BASED ON AGE AS WELL AS MOST ENGAGED**

In [51]:
def recommend_posts(user_id, top_n=5):
    # Content and collaborative scores
    user_profile = get_user_content_profile(user_id)
    content_scores = cosine_similarity(user_profile, post_vectors).flatten()

    post_ids = post_data['_id'].tolist()
    collab_scores = np.zeros(len(post_ids))  # Default for unmatched posts

    if user_id in interaction_matrix.index:
        user_index = interaction_matrix.index.get_loc(user_id)
        user_collab_scores = user_factors[user_index]
        collab_score_dict = dict(zip(interaction_matrix.columns, user_collab_scores))
        collab_scores = np.array([collab_score_dict.get(post_id, 0) for post_id in post_ids])

    # Normalize scores
    content_scores = content_scores / content_scores.max() if content_scores.max() > 0 else content_scores
    collab_scores = collab_scores / collab_scores.max() if collab_scores.max() > 0 else collab_scores
    popularity_scores = post_data['popularity_score'].values

    # Get user age group
    user_age_group = user_data.loc[user_data['_id'] == user_id, 'age_group'].values[0]

    # Weight posts based on user's age group
    age_weight = {
        'teen': 1.0,
        'young_adult': 1.2,
        'adult': 1.1,
        'senior': 0.9,
        'unknown': 1.0
    }
    age_group_weight = age_weight.get(user_age_group, 1.0)

    # Combine scores with age-based weight
    combined_scores = age_group_weight * (0.4 * content_scores + 0.3 * collab_scores + 0.3 * popularity_scores)

    # Exclude user's own posts
    user_posts = post_data[post_data['userId'] == user_id]['_id'].tolist()
    recommended_indices = [
        i for i in np.argsort(combined_scores)[-top_n * 2:][::-1]  # Select extra posts for filtering
        if post_ids[i] not in user_posts
    ][:top_n]

    return post_data.iloc[recommended_indices][['_id', 'postMessage', 'popularity_score']]

**ALOGIRTM WITH LESS WEIGHT FOR POPULARITY SCORES AND INCLUDING RANDOMNESS **

In [24]:
def recommend_posts(user_id, top_n=5):
    # Get user content profile
    user_profile = get_user_content_profile(user_id)
    content_scores = cosine_similarity(user_profile, post_vectors).flatten()

    post_ids = post_data['_id'].tolist()
    collab_scores = np.zeros(len(post_ids))

    if user_id in interaction_matrix.index:
        user_index = interaction_matrix.index.get_loc(user_id)
        user_collab_scores = user_factors[user_index]
        collab_score_dict = dict(zip(interaction_matrix.columns, user_collab_scores))
        collab_scores = np.array([collab_score_dict.get(post_id, 0) for post_id in post_ids])

    popularity_scores = post_data['popularity_score'].values

    # Normalize scores
    if content_scores.max() > 0:
        content_scores /= content_scores.max()
    if collab_scores.max() > 0:
        collab_scores /= collab_scores.max()
    if popularity_scores.max() > 0:
        popularity_scores = popularity_scores / popularity_scores.max()

    # Get age group weight
    user_age_group = user_data.loc[user_data['_id'] == user_id, 'age_group'].values[0] if user_id in user_data['_id'].values else 'unknown'
    age_weight = {'teen': 1.0, 'young_adult': 1.2, 'adult': 1.1, 'senior': 0.9, 'unknown': 1.0}
    age_group_weight = age_weight.get(user_age_group, 1.0)

    # Combine scores with randomness
    combined_scores = (
        0.5 * content_scores +
        0.35 * collab_scores +
        0.1 * popularity_scores +
        0.05 * age_group_weight +
        np.random.normal(0, 0.01, len(content_scores))  # Adding randomness
    )

    # Exclude user's own posts
    user_posts = post_data[post_data['userId'] == user_id]['_id'].tolist()
    recommended_indices = [
        i for i in np.argsort(combined_scores)[-top_n * 2:][::-1]
        if post_ids[i] not in user_posts
    ][:top_n]

    return post_data.iloc[recommended_indices][['_id', 'postMessage', 'popularity_score']]

In [25]:
user_id = input("Enter the User ID: ")
top_n = int(input("Enter the number of recommendations: "))

recommendations = recommend_posts(user_id, top_n)
print("Recommended Posts for User:")
print(recommendations)

Enter the User ID: 6246842ba1185e0d52b14468
Enter the number of recommendations: 5
Recommended Posts for User:
                         _id  \
12  64b92b56fc27b8ebe9da7fbd   
79  64b92b58fc27b8ebe9da86d4   
22  64b92b57fc27b8ebe9da8116   
89  64b92b59fc27b8ebe9da8a07   
76  64b92b58fc27b8ebe9da864e   

                                          postMessage  popularity_score  
12  If you could interview any cricketer, who woul...          1.000000  
79  This has to be the weirdest Dismissal in the h...          0.747664  
22  Twitter reacts to Ishan Kishan making his Test...          0.579439  
89                                  Zaheer Khan! GOD🙌          0.822430  
76  It's Guru Purnima today and here's looking at ...          0.579439  
